In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import win32com.client
import os
from datetime import datetime
from collections import OrderedDict
import time
import fitz
import string

x=0
dic_atual = os.getcwd()
nova_pasta = dic_atual+'\Arquivospdfs'

Excel = win32com.client.Dispatch("Excel.Application")
Workbook = Excel.ActiveWorkbook
Worksheet = Workbook.Worksheets("MCO")
letras = list(string.ascii_uppercase)[:26]

for letra in letras:
    if str(Worksheet.Range(f"{letra}1").Value)=="Viagem":
        valorf=2
        while True:
            if Worksheet.Range(f"{letra}{valorf}").Value==None:
                print(valorf)
                break
            valorf=valorf+1

vet = list(range(valorf))
vetlinha = list(range(valorf))
data_veiculo = list(range(valorf))
citgis = list(range(valorf))
codigo_viagem = list(range(valorf))
veiculo_data = list(range(valorf))
viagem = list(range(valorf))
veiculo_rep = list(range(valorf))

total_arq=0
y=0
for x in range(2, valorf):
    #motivo
    verificar = Worksheet.Range("BB" + str(x)).Value
    #citgis
    verificar2 = Worksheet.Range("AX" + str(x)).Value
    if verificar=='Viagem realizada com km abaixo do limite de 90% da extensão referencial (Art. 61, inciso IV do Decreto 18.370 de 7 de julho de 2023; Art. 7º da Portaria Nº 024/2023)' and verificar2!='':
        #veiculo
        vet[y] = str(Worksheet.Range("K" + str(x)).Value)
        vet[y] = vet[y].split('.')[0]
        #viagem
        codigo_viagem[y] = str(Worksheet.Range("W" + str(x)).Value)
        codigo_viagem[y] = codigo_viagem[y].split('.')[0]
        #citgis
        citgis[y] = str(Worksheet.Range("AX" + str(x)).Value)
        citgis[y] = citgis[y].split('.')[0]
        #Data
        data_veiculo[y] = str(Worksheet.Range("AR" + str(x)).Value)
        data_veiculo[y] = data_veiculo[y][8:10] + '/' + data_veiculo[y][5:7] + '/' + data_veiculo[y][:4]
        #linha
        vetlinha[y] = str(Worksheet.Range("F" + str(x)).Value)
        
        veiculo_data[y] = vet[y]+'-'+data_veiculo[y]
        veiculo_rep[y] = veiculo_data[y].replace("/", "-")
        #viagem
        viagem[y]=str(Worksheet.Range("W" + str(x)).Value)
        viagem[y]=viagem[y].split('.')[0]
        
        total_arq=total_arq+1
        y=y+1

lista_sem_repeticao = list(OrderedDict.fromkeys(veiculo_data))
total=0
list_t=0
for list in lista_sem_repeticao:
    list_t=list_t+1
print(list_t)
for x in range(0, list_t):
    teste=0

    for z in range(0, y):

        if lista_sem_repeticao[x]==veiculo_data[z] and teste==0:
            teste=1
            
    if teste==0:
        
        lista_sem_repeticao[x]=''
    else:
        total=total+1



options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
"download.default_directory": nova_pasta,
"download.prompt_for_download": False,
"download.directory_upgrade": True,
"safebrowsing.enabled": False,
"plugins.always_open_pdf_externally": True
})
options.add_argument("--safebrowsing-disable-download-protection")
options.add_argument("--disable-features=InfiniteSessionRestore")
options.add_argument("--disable-popup-blocking")
driver = webdriver.Chrome(options=options)

wait = WebDriverWait(driver, 10)

driver.get(r'http://citgis-next.sindpautras.org.br:8080/ecitbusSecurity/publico/login.jsp?logout')

time.sleep(3)
conta = driver.find_element('xpath', '//*[@id="usuario"]')
conta.send_keys('User')
senha = driver.find_element('xpath', '//*[@id="senha"]')
senha.send_keys('Senha')

entrar = driver.find_element('xpath', '//*[@id="botaoEntrar"]')
entrar.click()

time.sleep(2)
citimg = driver.find_element('xpath', '//*[@id="6"]/img')
citimg.click()

time.sleep(3)
driver.maximize_window()
menu = wait.until(EC.presence_of_element_located(('xpath', '//*[@id="menu_58"]/span')))
menu.click()

pontualidade = driver.find_element('xpath', '//*[@id="mnu_1088"]/li/a/div[2]')
pontualidade.click()



16318


In [2]:
for x in range(0, total):
    
    veic = lista_sem_repeticao[x].split('-')[0]
    data_t = lista_sem_repeticao[x].split('-')[1]
    junc = lista_sem_repeticao[x].replace("/", "-")
    time.sleep(1)
    limpar = wait.until(EC.presence_of_element_located(('xpath', '//*[@id="btnLimpaVeiculoRelatorio"]')))
    limpar.click()
    
    time.sleep(1)
    dat = driver.find_element('xpath', '//*[@id="relPontualidadePorVeiculoDataInicio"]')
    driver.execute_script("arguments[0].value = '';", dat)
    dat.send_keys(data_t)

    datf = driver.find_element('xpath', '//*[@id="relPontualidadePorVeiculoDataFinal"]')
    driver.execute_script("arguments[0].value = '';", datf)
    datf.send_keys(data_t)

    veiculo = driver.find_element('xpath', '//*[@id="relBaseFilterVeiculoDescricao"]')
    driver.execute_script("arguments[0].value = '';", veiculo)
    veiculo.send_keys(veic)
    
    time.sleep(1)
    veiculo.send_keys(Keys.ARROW_DOWN)
    time.sleep(1)
    veiculo.send_keys(Keys.ENTER)
    
    gerar = wait.until(EC.presence_of_element_located(('xpath', '//*[@id="btnGerarRel"]')))
    gerar.click()
    
    caminho_original = dic_atual+"\\Arquivospdfs\\relPontualidadePorVeiculo.pdf"
    caminho_destino = dic_atual+f"\\Arquivospdfs\\{junc}.pdf"
    
    contagem=0
    while True:
        time.sleep(1)
        if contagem==10:
            break
        try:
            os.rename(caminho_original, caminho_destino)
            break
        except FileNotFoundError:
            contagem=contagem+1
    if contagem==10:
        driver.switch_to.window(driver.window_handles[0])
        veiculo = driver.find_element('xpath', '//*[@id="btnVeiculoRelatorio"]')
        veiculo.click()
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[1])
        veiculo = driver.find_element('xpath', '//*[@id="veiculoNumero"]')
        time.sleep(1)
        veiculo.send_keys(Keys.CONTROL + 'a')
        veiculo.send_keys(Keys.BACKSPACE)
        veiculo.send_keys(veic)
        time.sleep(1)
        inativo = driver.find_element('xpath', '//*[@id="btnAbreFechaMaisFiltros"]')
        inativo.click()
        time.sleep(1)
        inativo = driver.find_element('xpath', '//*[@id="veiculoSituacao"]')
        time.sleep(1)
        inativo.send_keys(Keys.ARROW_DOWN)
        time.sleep(1)
        inativo.send_keys(Keys.ENTER)
        veiculo.send_keys(Keys.ENTER)
        time.sleep(1)
        contagem=0
        while True:
            time.sleep(1)
            if contagem==10:
                print(f"{veic} Não está no sistema")
                break
            try:
                inativo = driver.find_element('xpath', '//*[@id="tableVeiculo"]/tbody/tr/td[2]')
            except NoSuchElementException:
                contagem=contagem+1
        if contagem==10:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue
        inativo.click()
        driver.switch_to.window(driver.window_handles[0])
        gerar.click()
        contagem=0
        while True:
            time.sleep(1)
            if contagem==10:
                print(f"{veic} Não está no sistema")
                break
            try:
                os.rename(caminho_original, caminho_destino)
                break
            except FileNotFoundError:
                contagem=contagem+1
    
driver.quit()
print('Todos PDFs criados com sucesso!!!')

diretorio = dic_atual+'\\Arquivospdfs'
total_pdfs = 0

for nome_arquivo in os.listdir(diretorio):
    if nome_arquivo.endswith('.pdf'):
        total_pdfs += 1

for z in range(0, total_arq-1):
    try:
        pdf = fitz.open(dic_atual+f"\\Arquivospdfs\\{veiculo_rep[z]}.pdf")
        total_paginas = pdf.page_count


        for pa in range(0, total_paginas-1):
            pagina=pdf[pa]
            ocorrencias = pagina.search_for('Cód. Viagem '+citgis[z])

            if ocorrencias!=[]:
                for ocorr in ocorrencias:
                    annot = pagina.add_highlight_annot(ocorr, (1, 0, 0))
                    annot.update()

                nova_pasta = dic_atual+f"\\dec\\{vetlinha[z]}\\Abaixo de 90\\{viagem[z]}"

                if not os.path.exists(nova_pasta):
                    os.makedirs(nova_pasta)
                else:
                    print(f"A pasta '{nova_pasta}' já existe.")

                pdf.save(dic_atual+f"\\dec\\{vetlinha[z]}\\Abaixo de 90\\{viagem[z]}\\relPontualidadePorVeiculo {viagem[z]}.pdf")
    except FileNotFoundError:
        print("Erros")
        print(f"{veiculo_rep[z]}")
        continue
    except Exception as e:
        print("Erros")
        print(f"{veiculo_rep[z]}")
        continue
print('Concluido!!')


Todos PDFs criados com sucesso!!!
Concluido!!
